In [3]:
import os
from constants import huggingfaceHub_api_token

os.environ['HUGGINGFACEHUB_API_TOKEN']=huggingfaceHub_api_token

In [2]:
from langchain import HuggingFaceHub
llm=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

d:\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
d:\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
output=llm.predict("What is the capital of Russia")
output

'moscow'

<!-- Prompt Templates -->

In [18]:
from langchain.prompts import PromptTemplate

prompt=PromptTemplate(input_variables=['country'],
                      template="Tell me the capital of {country}")

prompt.format(country="India")

'Tell me the capital of India'

In [19]:
from langchain.chains import LLMChain

chain=LLMChain(llm=llm,prompt=prompt)
chain.run("India")

'calcutta'

In [20]:
prompt=PromptTemplate(input_variables=['country'],
                      template="The capital of India is New Delhi. Tell me the capital of {country}")

prompt.format(country="India")

'The capital of India is New Delhi. Tell me the capital of India'

In [21]:
chain=LLMChain(llm=llm,prompt=prompt)
chain.run("India")

'New Delhi'

<!-- Combining Multiple chains Simple sequential chain -->

In [22]:
capital_prompt=PromptTemplate(input_variables=['country'],
                      template="Tell me the capital of {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_prompt)

In [29]:
place_prompt=PromptTemplate(input_variables=['capital'],
                      template="Tell me one famous place located in {capital}")

place_chain=LLMChain(llm=llm,prompt=place_prompt)

In [30]:
from langchain.chains import SimpleSequentialChain

chain=SimpleSequentialChain(chains=[capital_chain,place_chain])
chain.run("India")

'narayan temple'

<!-- sequential chain -->

In [31]:
capital_prompt=PromptTemplate(input_variables=['country'],
                      template="Tell me the capital of {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_prompt,output_key="capital")

In [32]:
place_prompt=PromptTemplate(input_variables=['capital'],
                      template="Tell me one famous place located in {capital}")

place_chain=LLMChain(llm=llm,prompt=place_prompt,output_key="place")

In [35]:
from langchain.chains import SequentialChain

chain=SequentialChain(chains=[capital_chain,place_chain],
                      input_variables=['country'],
                      output_variables=['capital','place'])

chain({'country':"India"})

{'country': 'India', 'capital': 'calcutta', 'place': 'narayan temple'}

<!-- Chat model using OpenAI -->

In [18]:
from constants import openai_key
# from langchain.chat_models import ChatOllama
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

os.environ["OPENAI_API_KEY"]=openai_key

In [19]:
# chatllm=ChatOllama(temperature=0.6,model='llama3',ollama_server_url = "http://localhost:11434")
chatllm=ChatOpenAI(temperature=0.6)
chatllm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F4A4C0D490>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001F4A4C0BC80>, temperature=0.6, openai_api_key='sk-proj-rnF4WjLqwL11szgGOiWnT3BlbkFJhKm8xGRzBFJgUEffhsLj', openai_proxy='')

In [20]:
chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines in AI")
])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

<!--Prompt Template+LLM+Ouput Parsers  -->

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [22]:
class Commasepratedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [23]:
template="You are a helpful assistant. When a user gives any input, you should generate 5 words synonyms in a comma separated manner."
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [24]:
chain=chatprompt|chatllm|Commasepratedoutput()

In [25]:
chain.invoke({"text":"intelligent"})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}